# 라이브러리 불러오기

In [65]:
import numpy as np
import pandas as pd
import json

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

from collections import defaultdict
from itertools import chain, combinations
from optparse import OptionParser

In [2]:
# !pip install mlxtend

In [16]:
?apriori

In [7]:
sample = [["사과", "맥주", "치킨", "쌀"], ["사과", "맥주", "쌀"], ["사과", "맥주"], 
          ["사과", "바나나"], ["맥주", "치킨", "우유", "쌀"], ["맥주", "우유", "쌀"],
          ["맥주", "우유"], ["사과", "바나나"]]

In [11]:
te = TransactionEncoder()
te_result = te.fit(sample).transform(sample)

In [13]:
te_result

array([[ True, False,  True,  True, False,  True],
       [ True, False,  True,  True, False, False],
       [ True, False,  True, False, False, False],
       [False,  True,  True, False, False, False],
       [ True, False, False,  True,  True,  True],
       [ True, False, False,  True,  True, False],
       [ True, False, False, False,  True, False],
       [False,  True,  True, False, False, False]])

In [17]:
sample_df = pd.DataFrame(te_result, columns=["사과", "바나나", "맥주", "치킨", "우유", "쌀"])

In [18]:
sample_df

,사과,바나나,맥주,치킨,우유,쌀
0,True,False,True,True,False,True
1,True,False,True,True,False,False
2,True,False,True,False,False,False
3,False,True,True,False,False,False
4,True,False,False,True,True,True
5,True,False,False,True,True,False
6,True,False,False,False,True,False
7,False,True,True,False,False,False


In [23]:
itemsets = apriori(sample_df, min_support=0.5, use_colnames=True)
itemsets

,support,itemsets
0,0.750,(사과)
1,0.625,(맥주)
2,0.500,(치킨)
3,0.500,"(사과, 치킨)"


In [31]:
association_rules(itemsets, metric="confidence", min_threshold=0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사과),(치킨),0.75,0.50,0.5,0.666667,1.333333,0.125,1.5
1,(치킨),(사과),0.50,0.75,0.5,1.000000,1.333333,0.125,inf


# 레시피 json 파일 열기

In [66]:
with open('Final/Deduplicated_Filtered_recipe_data_Final.json', encoding="utf-8") as f:
    data = json.load(f)

# 데이터 확인

In [67]:
data

{'data': [{'code': 0,
   'title': '맛살볶음밥',
   'ingredients': ['맛살',
    '맛술',
    '후춧가루',
    '간장',
    '즉석밥',
    '소금',
    '다진마늘',
    '멸치액젓',
    '대파',
    '설탕',
    '계란'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1750'},
  {'code': 1,
   'title': '돈가스 김밥 샌드위치',
   'ingredients': ['청피망',
    '당근',
    '김',
    '단무지',
    '즉석밥',
    '소금',
    '참기름',
    '깻잎',
    '돈가스소스',
    '돈가스'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1625'},
  {'code': 2,
   'title': '묵은지말이 참치김밥',
   'ingredients': ['참깨', '즉석밥', '묵은지', '마요네즈', '소금', '통조림참치', '참기름'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1628'},
  {'code': 3,
   'title': '마늘종장아찌 볶음밥',
   'ingredients': ['후춧가루', '즉석밥', '마늘', '참기름', '스팸', '마늘종장아찌', '계란'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1754'},
  {'code': 4,
   'title': '새우볶음밥',
   'ingredients': ['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파', '계란'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTh

In [68]:
df_raw = pd.DataFrame(data["data"])

In [69]:
df_raw

,code,title,ingredients,url
0,0,맛살볶음밥,"[맛살, 맛술, 후춧가루, 간장, 즉석밥, 소금, 다진마늘, 멸치액젓, 대파, 설탕...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,1,돈가스 김밥 샌드위치,"[청피망, 당근, 김, 단무지, 즉석밥, 소금, 참기름, 깻잎, 돈가스소스, 돈가스]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,2,묵은지말이 참치김밥,"[참깨, 즉석밥, 묵은지, 마요네즈, 소금, 통조림참치, 참기름]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,3,마늘종장아찌 볶음밥,"[후춧가루, 즉석밥, 마늘, 참기름, 스팸, 마늘종장아찌, 계란]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
4,4,새우볶음밥,"[냉동새우, 후춧가루, 간장, 즉석밥, 소금, 마늘, 대파, 계란]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
...,...,...,...,...
1665,2908,퓨전라따뚜이,"[토마토소스, 파라타, 방울토마토, 바질가루, 주키니호박, 올리브유, 가지, 노란파...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
1666,2910,누룽지 카레 그라탱,"[누룽지, 크림수프가루, 카레가루, 슈레드모차렐라치즈, 파슬리]",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
1667,2911,감자베이컨스튜,"[감자, 닭육수, 양파, 베이컨, 파슬리, 타임, 쪽파, 셀러리, 대파, 백태, 양배추]",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
1668,2912,싸 먹는 비빔떡,"[참깨, 고추장, 계란, 식초, 당근, 간장, 소금, 무, 물엿, 밀가루, 다진마늘...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...


# 사용자가 입력한 재료가 포함된 레시피만 가져오기

In [119]:
user_raw = input("재료를 소금, 간장과 같은 형태로 입력하세요 : ")

재료를 소금, 간장과 같은 형태로 입력하세요 : 김치, 대파


In [120]:
user = user_raw.split(", ")
user

['김치', '대파']

In [122]:
user_lst = []
for i in range(len(df_raw)):
    if(user[0] in df_raw.iloc[i]["ingredients"] and user[1] in df_raw.iloc[i]["ingredients"]):
        user_lst.append(tuple(df_raw.iloc[i]))

In [123]:
user_df = pd.DataFrame(user_lst, columns=["code", "title", "ingredients", "url"])
user_df

,code,title,ingredients,url
0,84,불고기 토핑밥,"['식초', '양파', '후춧가루', '김치', '간장', '들기름', '즉석밥',...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,122,제육양념밥,"['맛술', '후춧가루', '김치', '간장', '고추장', '돼지고기앞다리살', ...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,143,불고기 김치콩나물밥,"['콩나물', '맛술', '후춧가루', '김치', '간장', '참깨', '소고기안심...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,377,얼큰 김치우동,"['후춧가루', '김치', '유부', '우동사리', '꼬치어묵', '대파']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
4,394,참치김치찌개,"['양파', '김치', '통조림참치', '고추장', '두부', '대파', '설탕']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
5,401,만두전골,"['국간장', '양파', '느타리버섯', '참깨', '고춧가루', '만두피', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
6,411,새우젓비지찌개,"['김치', '들기름', '새우젓', '청양고추', '다진마늘', '대파', '백태']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
7,412,차돌박이청국장,"['청국장', '양파', '국물용멸치', '김치', '홍고추', '고춧가루', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
8,413,두부김치찌개,"['국간장', '양파', '국물용멸치', '들기름', '김치', '고춧가루', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
9,416,쌈장목살찌개,"['감자', '생강', '후춧가루', '김치', '청주', '소금', '청양고추',...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...


In [75]:
user_df.to_excel('user_df.xlsx')

In [76]:
df = pd.read_excel('user_df.xlsx')

In [77]:
df = df.drop(["Unnamed: 0"], axis=1)

In [78]:
# df = df.drop_duplicates()
df

,code,title,ingredients,url
0,84,불고기 토핑밥,"['식초', '양파', '후춧가루', '김치', '간장', '들기름', '즉석밥',...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,122,제육양념밥,"['맛술', '후춧가루', '김치', '간장', '고추장', '돼지고기앞다리살', ...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,143,불고기 김치콩나물밥,"['콩나물', '맛술', '후춧가루', '김치', '간장', '참깨', '소고기안심...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,377,얼큰 김치우동,"['후춧가루', '김치', '유부', '우동사리', '꼬치어묵', '대파']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
4,394,참치김치찌개,"['양파', '김치', '통조림참치', '고추장', '두부', '대파', '설탕']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
5,401,만두전골,"['국간장', '양파', '느타리버섯', '참깨', '고춧가루', '만두피', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
6,411,새우젓비지찌개,"['김치', '들기름', '새우젓', '청양고추', '다진마늘', '대파', '백태']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
7,412,차돌박이청국장,"['청국장', '양파', '국물용멸치', '김치', '홍고추', '고춧가루', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
8,413,두부김치찌개,"['국간장', '양파', '국물용멸치', '들기름', '김치', '고춧가루', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
9,416,쌈장목살찌개,"['감자', '생강', '후춧가루', '김치', '청주', '소금', '청양고추',...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...


In [79]:
# df_raw = df.reset_index(drop=True)
df_raw = df
df_raw

,code,title,ingredients,url
0,84,불고기 토핑밥,"['식초', '양파', '후춧가루', '김치', '간장', '들기름', '즉석밥',...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,122,제육양념밥,"['맛술', '후춧가루', '김치', '간장', '고추장', '돼지고기앞다리살', ...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,143,불고기 김치콩나물밥,"['콩나물', '맛술', '후춧가루', '김치', '간장', '참깨', '소고기안심...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,377,얼큰 김치우동,"['후춧가루', '김치', '유부', '우동사리', '꼬치어묵', '대파']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
4,394,참치김치찌개,"['양파', '김치', '통조림참치', '고추장', '두부', '대파', '설탕']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
5,401,만두전골,"['국간장', '양파', '느타리버섯', '참깨', '고춧가루', '만두피', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
6,411,새우젓비지찌개,"['김치', '들기름', '새우젓', '청양고추', '다진마늘', '대파', '백태']",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
7,412,차돌박이청국장,"['청국장', '양파', '국물용멸치', '김치', '홍고추', '고춧가루', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
8,413,두부김치찌개,"['국간장', '양파', '국물용멸치', '들기름', '김치', '고춧가루', '소...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...
9,416,쌈장목살찌개,"['감자', '생강', '후춧가루', '김치', '청주', '소금', '청양고추',...",https://2bob.co.kr/recipe.php?id=view&eTheme=3...


# 재료 원핫인코딩

In [80]:
df_raw["ingredients"]

0     ['식초', '양파', '후춧가루', '김치', '간장', '들기름', '즉석밥',...
1     ['맛술', '후춧가루', '김치', '간장', '고추장', '돼지고기앞다리살', ...
2     ['콩나물', '맛술', '후춧가루', '김치', '간장', '참깨', '소고기안심...
3            ['후춧가루', '김치', '유부', '우동사리', '꼬치어묵', '대파']
4        ['양파', '김치', '통조림참치', '고추장', '두부', '대파', '설탕']
5     ['국간장', '양파', '느타리버섯', '참깨', '고춧가루', '만두피', '소...
6      ['김치', '들기름', '새우젓', '청양고추', '다진마늘', '대파', '백태']
7     ['청국장', '양파', '국물용멸치', '김치', '홍고추', '고춧가루', '소...
8     ['국간장', '양파', '국물용멸치', '들기름', '김치', '고춧가루', '소...
9     ['감자', '생강', '후춧가루', '김치', '청주', '소금', '청양고추',...
10    ['양파', '김치', '국물용멸치', '들기름', '다시마', '쌈장', '두부'...
11    ['국간장', '양파', '김치', '고춧가루', '순두부', '통조림참치', '다...
12    ['생강', '후춧가루', '김치', '고춧가루', '새우젓', '청주', '소금'...
13    ['비엔나소시지', '김치', '간장', '참기름', '고추장', '깻잎', '라면...
14    ['생강', '양파', '즉석밥', '고춧가루', '고추장', '간장', '닭다리살...
15    ['참깨', '즉석밥', '매실청', '고춧가루', '고추장', '대패삼겹살', '...
16    ['삼겹살', '생강', '양파', '김치', '국물용멸치', '홍고추', '청주'...
17    ['감자', '만두', '국간장', '양파', '국물용멸치', '김치', '

In [81]:
type(df["ingredients"][1])

str

### 재료가 문자열 형태, 오류 발생

In [82]:
import ast
list_in_str = df_raw["ingredients"][1]
list_data = ast.literal_eval(list_in_str)
print(type(list_data))

<class 'list'>


In [83]:
list_data

['맛술', '후춧가루', '김치', '간장', '고추장', '돼지고기앞다리살', '다진마늘', '쌀', '대파', '올리고당']

In [84]:
list_data = []
for i in range(len(df_raw["ingredients"])):
    list_data.append(ast.literal_eval(df_raw["ingredients"][i]))

In [85]:
type(list_data)

list

In [86]:
data_en = []
for i in list_data:
    data_en.append(i)

In [87]:
data_en

[['식초',
  '양파',
  '후춧가루',
  '김치',
  '간장',
  '들기름',
  '즉석밥',
  '영양부추',
  '참깨',
  '소금',
  '참기름',
  '고추장',
  '연겨자',
  '다진마늘',
  '소고기불고기용',
  '대파',
  '설탕'],
 ['맛술', '후춧가루', '김치', '간장', '고추장', '돼지고기앞다리살', '다진마늘', '쌀', '대파', '올리고당'],
 ['콩나물',
  '맛술',
  '후춧가루',
  '김치',
  '간장',
  '참깨',
  '소고기안심',
  '참기름',
  '다진마늘',
  '쌀',
  '대파',
  '설탕'],
 ['후춧가루', '김치', '유부', '우동사리', '꼬치어묵', '대파'],
 ['양파', '김치', '통조림참치', '고추장', '두부', '대파', '설탕'],
 ['국간장',
  '양파',
  '느타리버섯',
  '참깨',
  '고춧가루',
  '만두피',
  '소금',
  '마늘',
  '다시마',
  '소고기양지머리',
  '풋고추',
  '미나리',
  '후춧가루',
  '숙주나물',
  '다진마늘',
  '표고버섯',
  '두부',
  '대파',
  '김치',
  '홍고추',
  '참기름',
  '다진소고기'],
 ['김치', '들기름', '새우젓', '청양고추', '다진마늘', '대파', '백태'],
 ['청국장',
  '양파',
  '국물용멸치',
  '김치',
  '홍고추',
  '고춧가루',
  '소고기차돌박이',
  '무',
  '애호박',
  '청양고추',
  '두부',
  '다진마늘',
  '대파'],
 ['국간장',
  '양파',
  '국물용멸치',
  '들기름',
  '김치',
  '고춧가루',
  '소금',
  '고추장',
  '다시마',
  '된장',
  '다진마늘',
  '두부',
  '대파'],
 ['감자', '생강', '후춧가루', '김치', '청주', '소금', '청양고추', '쌈장', '두부', '돼지고기목살', '대파'],
 ['

In [88]:
te = TransactionEncoder()
te_ary = te.fit(data_en).transform(data_en)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [89]:
df

,간장,감자,고구마,고추장,고춧가루,국간장,국물용멸치,김가루,김치,깻잎,...,청주,칼국수면,콩나물,콩비지,통조림참치,표고버섯,풋고추,햄,홍고추,후춧가루
0,True,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
1,True,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
2,True,False,False,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
5,False,False,False,False,True,True,False,False,True,False,...,False,False,False,False,False,True,True,False,True,True
6,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,True,False
8,False,False,False,True,True,True,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
9,False,True,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,True


# support 탐색

In [90]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent = apriori(df, min_support=0.1, use_colnames=True, verbose=True)
frequent
frequent.sort_values("support", ascending=False)

Processing 168 combinations | Sampling itemset size 8


,support,itemsets
75,1.000000,"(김치, 대파)"
9,1.000000,(대파)
5,1.000000,(김치)
8,0.571429,(다진마늘)
293,0.571429,"(다진마늘, 김치, 대파)"
...,...,...
17,0.142857,(즉석밥)
471,0.142857,"(참기름, 김치, 소금, 간장)"
472,0.142857,"(김치, 소금, 간장, 참깨)"
473,0.142857,"(참기름, 김치, 즉석밥, 간장)"


In [91]:
# pd.set_option('display.max_rows', 10000)

In [92]:
features = df.astype(float)
features

,간장,감자,고구마,고추장,고춧가루,국간장,국물용멸치,김가루,김치,깻잎,...,청주,칼국수면,콩나물,콩비지,통조림참치,표고버섯,풋고추,햄,홍고추,후춧가루
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# 레시피 제목 추출

In [93]:
labels = df_raw["title"]
labels

0        불고기 토핑밥
1          제육양념밥
2     불고기 김치콩나물밥
3        얼큰 김치우동
4         참치김치찌개
5           만두전골
6        새우젓비지찌개
7        차돌박이청국장
8         두부김치찌개
9         쌈장목살찌개
10        쌈장김치찌개
11       순두부김치찌개
12          비지찌개
13         부대라볶이
14           닭갈비
15            콩불
16       삼겹살묵은지찜
17         얼큰만제비
18        김치스팸라면
19         김치칼국수
20          메밀전병
Name: title, dtype: object

In [94]:
### test
# lst = []
# for i in labels:
#     lst.append(i)
    
# lst

# Apriori Algorithm

In [95]:
def powerset(s): # 멱집합
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))


def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet


def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet


def associationRule(freqItemSet, recipes, labels, itemSetWithSup, minConf):
    rules = []
    lbls = []
    
    sets_compare = [set(re) for re in recipes]
    
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            
            for i in labels:
                lbls.append(i)
            
            for s in subsets:
                
                
                ### 기존에 두개의 레시피만 가져오게 되어 있어서 레시피 제목을 두개만 받아오는 문제 발생
#                 ss = set([es for es in s])
#                 if(len(ss.intersection(sets_compare[0])) > 0 and len(ss.intersection(sets_compare[1])) > 0):
#                     lbls.append([labels[0], labels[1]])
#                 if(len(ss.intersection(sets_compare[0])) > 0):
#                     lbls.append(labels[0])
#                 if(len(ss.intersection(sets_compare[1])) > 0):
#                     lbls.append(labels[1])

                    
                    
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules, lbls


def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

In [96]:
def apriori(itemSetList, labels, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules, lbls = associationRule(globalFreqItemSet, itemSetList, labels, globalItemSetWithSup, minConf)
    #rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules, lbls

In [97]:
idx_to_column = {k:v for k,v in enumerate(features.columns.values)}
column_to_idx = {v:k for k,v in enumerate(features.columns.values)}

In [98]:
def get_random_recipes(features, labels, nr):
    ids = np.random.randint(0, len(features), size=nr).tolist()
    idxs = np.array([np.arange(len(features.columns.values)).tolist() for x in range(len(labels))])
    multiply = features.iloc[:].values.astype(np.int32) * idxs
    idx_arr  = [np.where(mult != 0)[0].tolist() for mult in multiply]
    recipes = []
    for arr_idx in idx_arr:
        temp_result = []
        for idx in arr_idx:
            temp_result.append(idx_to_column[idx])
        recipes.append(temp_result)
    return recipes, labels[:].values

In [99]:
def print_recipe(recipe, label):
    print("-" * 50)
    print("Recipe: ", label)
    print("-" * 50)
    for item in recipe:
        print(item)

In [104]:
# get 2 random receipes
recipes, lbls = get_random_recipes(features, labels, nr=5)

# print recipes
pr = [print_recipe(rec, lbl) for rec, lbl in list(zip(recipes,lbls))]

--------------------------------------------------
Recipe:  불고기 토핑밥
--------------------------------------------------
고추장
김치
다진마늘
대파
들기름
설탕
소고기불고기용
소금
식초
양파
연겨자
영양부추
즉석밥
참기름
참깨
후춧가루
--------------------------------------------------
Recipe:  제육양념밥
--------------------------------------------------
고추장
김치
다진마늘
대파
돼지고기앞다리살
맛술
쌀
올리고당
후춧가루
--------------------------------------------------
Recipe:  불고기 김치콩나물밥
--------------------------------------------------
김치
다진마늘
대파
맛술
설탕
소고기안심
쌀
참기름
참깨
콩나물
후춧가루
--------------------------------------------------
Recipe:  얼큰 김치우동
--------------------------------------------------
김치
꼬치어묵
대파
우동사리
유부
후춧가루
--------------------------------------------------
Recipe:  참치김치찌개
--------------------------------------------------
고추장
김치
대파
두부
설탕
양파
통조림참치
--------------------------------------------------
Recipe:  만두전골
--------------------------------------------------
고춧가루
국간장
김치
느타리버섯
다시마
다진마늘
다진소고기
대파
두부
마늘
만두피
미나리
소고기양지머리
소금
숙주나물
양파
참기름
참깨
표고버섯
풋고추
홍고추
후춧가루
--

In [105]:
freqItemSet, rules, lbls = apriori(recipes, lbls, minSup=0.25, minConf=0.25)

In [106]:
freqItemSet

{1: {frozenset({'고추장'}),
  frozenset({'고춧가루'}),
  frozenset({'참기름'}),
  frozenset({'소금'}),
  frozenset({'김치'}),
  frozenset({'설탕'}),
  frozenset({'후춧가루'}),
  frozenset({'대파'}),
  frozenset({'양파'}),
  frozenset({'다진마늘'}),
  frozenset({'두부'})},
 2: {frozenset({'김치', '대파'}),
  frozenset({'대파', '소금'}),
  frozenset({'고추장', '김치'}),
  frozenset({'다진마늘', '대파'}),
  frozenset({'고춧가루', '김치'}),
  frozenset({'대파', '참기름'}),
  frozenset({'대파', '설탕'}),
  frozenset({'대파', '후춧가루'}),
  frozenset({'고춧가루', '다진마늘'}),
  frozenset({'고춧가루', '대파'}),
  frozenset({'고추장', '대파'}),
  frozenset({'다진마늘', '설탕'}),
  frozenset({'다진마늘', '후춧가루'}),
  frozenset({'대파', '두부'}),
  frozenset({'김치', '참기름'}),
  frozenset({'김치', '설탕'}),
  frozenset({'김치', '후춧가루'}),
  frozenset({'대파', '양파'}),
  frozenset({'김치', '다진마늘'}),
  frozenset({'김치', '양파'}),
  frozenset({'김치', '두부'}),
  frozenset({'설탕', '참기름'}),
  frozenset({'김치', '소금'}),
  frozenset({'다진마늘', '양파'}),
  frozenset({'고춧가루', '양파'})},
 3: {frozenset({'김치', '대파', '소금'}),
  frozenset

In [124]:
rules

[[{'김치'}, {'대파'}, 1.0],
 [{'대파'}, {'김치'}, 1.0],
 [{'대파'}, {'소금'}, 0.38095238095238093],
 [{'소금'}, {'대파'}, 1.0],
 [{'김치'}, {'고추장'}, 0.3333333333333333],
 [{'고추장'}, {'김치'}, 1.0],
 [{'다진마늘'}, {'대파'}, 1.0],
 [{'대파'}, {'다진마늘'}, 0.5714285714285714],
 [{'김치'}, {'고춧가루'}, 0.38095238095238093],
 [{'고춧가루'}, {'김치'}, 1.0],
 [{'참기름'}, {'대파'}, 1.0],
 [{'대파'}, {'참기름'}, 0.42857142857142855],
 [{'대파'}, {'설탕'}, 0.42857142857142855],
 [{'설탕'}, {'대파'}, 1.0],
 [{'대파'}, {'후춧가루'}, 0.42857142857142855],
 [{'후춧가루'}, {'대파'}, 1.0],
 [{'다진마늘'}, {'고춧가루'}, 0.5833333333333334],
 [{'고춧가루'}, {'다진마늘'}, 0.875],
 [{'대파'}, {'고춧가루'}, 0.38095238095238093],
 [{'고춧가루'}, {'대파'}, 1.0],
 [{'대파'}, {'고추장'}, 0.3333333333333333],
 [{'고추장'}, {'대파'}, 1.0],
 [{'다진마늘'}, {'설탕'}, 0.5],
 [{'설탕'}, {'다진마늘'}, 0.6666666666666666],
 [{'다진마늘'}, {'후춧가루'}, 0.5],
 [{'후춧가루'}, {'다진마늘'}, 0.6666666666666666],
 [{'대파'}, {'두부'}, 0.2857142857142857],
 [{'두부'}, {'대파'}, 1.0],
 [{'참기름'}, {'김치'}, 1.0],
 [{'김치'}, {'참기름'}, 0.42857142857142855],
 [{'김치'}, {'설탕'},

In [125]:
lbls

['불고기 토핑밥',
 '제육양념밥',
 '불고기 김치콩나물밥',
 '얼큰 김치우동',
 '참치김치찌개',
 '만두전골',
 '새우젓비지찌개',
 '차돌박이청국장',
 '두부김치찌개',
 '쌈장목살찌개',
 '쌈장김치찌개',
 '순두부김치찌개',
 '비지찌개',
 '부대라볶이',
 '닭갈비',
 '콩불',
 '삼겹살묵은지찜',
 '얼큰만제비',
 '김치스팸라면',
 '김치칼국수',
 '메밀전병',
 '불고기 토핑밥',
 '제육양념밥',
 '불고기 김치콩나물밥',
 '얼큰 김치우동',
 '참치김치찌개',
 '만두전골',
 '새우젓비지찌개',
 '차돌박이청국장',
 '두부김치찌개',
 '쌈장목살찌개',
 '쌈장김치찌개',
 '순두부김치찌개',
 '비지찌개',
 '부대라볶이',
 '닭갈비',
 '콩불',
 '삼겹살묵은지찜',
 '얼큰만제비',
 '김치스팸라면',
 '김치칼국수',
 '메밀전병',
 '불고기 토핑밥',
 '제육양념밥',
 '불고기 김치콩나물밥',
 '얼큰 김치우동',
 '참치김치찌개',
 '만두전골',
 '새우젓비지찌개',
 '차돌박이청국장',
 '두부김치찌개',
 '쌈장목살찌개',
 '쌈장김치찌개',
 '순두부김치찌개',
 '비지찌개',
 '부대라볶이',
 '닭갈비',
 '콩불',
 '삼겹살묵은지찜',
 '얼큰만제비',
 '김치스팸라면',
 '김치칼국수',
 '메밀전병',
 '불고기 토핑밥',
 '제육양념밥',
 '불고기 김치콩나물밥',
 '얼큰 김치우동',
 '참치김치찌개',
 '만두전골',
 '새우젓비지찌개',
 '차돌박이청국장',
 '두부김치찌개',
 '쌈장목살찌개',
 '쌈장김치찌개',
 '순두부김치찌개',
 '비지찌개',
 '부대라볶이',
 '닭갈비',
 '콩불',
 '삼겹살묵은지찜',
 '얼큰만제비',
 '김치스팸라면',
 '김치칼국수',
 '메밀전병',
 '불고기 토핑밥',
 '제육양념밥',
 '불고기 김치콩나물밥',
 '얼큰 김치우동',
 '참치김치찌개',
 '만두전골',
 '새우젓비지찌개',
 '차돌박이청국장',
 '두부김치찌개',


In [108]:
recipes

[['고추장',
  '김치',
  '다진마늘',
  '대파',
  '들기름',
  '설탕',
  '소고기불고기용',
  '소금',
  '식초',
  '양파',
  '연겨자',
  '영양부추',
  '즉석밥',
  '참기름',
  '참깨',
  '후춧가루'],
 ['고추장', '김치', '다진마늘', '대파', '돼지고기앞다리살', '맛술', '쌀', '올리고당', '후춧가루'],
 ['김치', '다진마늘', '대파', '맛술', '설탕', '소고기안심', '쌀', '참기름', '참깨', '콩나물', '후춧가루'],
 ['김치', '꼬치어묵', '대파', '우동사리', '유부', '후춧가루'],
 ['고추장', '김치', '대파', '두부', '설탕', '양파', '통조림참치'],
 ['고춧가루',
  '국간장',
  '김치',
  '느타리버섯',
  '다시마',
  '다진마늘',
  '다진소고기',
  '대파',
  '두부',
  '마늘',
  '만두피',
  '미나리',
  '소고기양지머리',
  '소금',
  '숙주나물',
  '양파',
  '참기름',
  '참깨',
  '표고버섯',
  '풋고추',
  '홍고추',
  '후춧가루'],
 ['김치', '다진마늘', '대파', '들기름', '백태', '새우젓', '청양고추'],
 ['고춧가루',
  '국물용멸치',
  '김치',
  '다진마늘',
  '대파',
  '두부',
  '무',
  '소고기차돌박이',
  '애호박',
  '양파',
  '청국장',
  '청양고추',
  '홍고추'],
 ['고추장',
  '고춧가루',
  '국간장',
  '국물용멸치',
  '김치',
  '다시마',
  '다진마늘',
  '대파',
  '된장',
  '두부',
  '들기름',
  '소금',
  '양파'],
 ['감자', '김치', '대파', '돼지고기목살', '두부', '생강', '소금', '쌈장', '청양고추', '청주', '후춧가루'],
 ['국물용멸치', '김치', '다시마', '대파', '두부', '들기름', '쌈

## 유저가 고른 두 개의 재료와 연관성이 높은 재료가 들어간 레시피 찾기

In [109]:
user_ing = set(user)
additional_ing = set()
for i in rules:
    if(user_ing == i[0]):
        for j in i[1]:
            additional_ing.add(j)

In [110]:
additional = list(additional_ing)
additional

['참기름', '양파', '고춧가루', '다진마늘', '고추장', '설탕', '두부', '소금', '후춧가루']

In [111]:

add_recipe = []

for i in additional:
    add_ing = []
    add_ing = list(user_ing)
    add_ing.append(i)
    print(add_ing)
    for j, re in enumerate(recipes):
        if (add_ing[0] in re and add_ing[1] in re and add_ing[2]):
            add_recipe.append(j)

['김치', '대파', '참기름']
['김치', '대파', '양파']
['김치', '대파', '고춧가루']
['김치', '대파', '다진마늘']
['김치', '대파', '고추장']
['김치', '대파', '설탕']
['김치', '대파', '두부']
['김치', '대파', '소금']
['김치', '대파', '후춧가루']


In [112]:
print(len(add_recipe))

add_recipe = set(add_recipe)
add_recipe

print(len(add_recipe))

for i in add_recipe:
    print(lbls[i])

189
21
불고기 토핑밥
제육양념밥
불고기 김치콩나물밥
얼큰 김치우동
참치김치찌개
만두전골
새우젓비지찌개
차돌박이청국장
두부김치찌개
쌈장목살찌개
쌈장김치찌개
순두부김치찌개
비지찌개
부대라볶이
닭갈비
콩불
삼겹살묵은지찜
얼큰만제비
김치스팸라면
김치칼국수
메밀전병


In [113]:
def find_many_recipe(ingredients, rules, recipes, lbls):
    
    user_ing = set(ingredients)
    additional_ing = set()
    for i in rules:
        if(user_ing == i[0]): # 만약 유저가 입력한 재료가 규칙 첫번째 원소라면 [{"계란", "소금"}, {}]
            for j in i[1]: # 유저가 입력한 재료와 연관성이 높은 재료를 set에 추가(중복 없이)
                additional_ing.add(j)
                
    additional = list(additional_ing) # 중복 없이 추가한 연관성 높은 재료를 리스트화
    
    add_recipe = []

    for i in additional: # 연관성 높은 재료 하나씩
        add_ing = []
        add_ing = list(user_ing) # 유저가 입력한 재료를 리스트화(set으로 바꾼 것을 다시 변경)
        add_ing.append(i) # 유저가 입력한 재료 리스트에 연관성 높은 재료를 추가(유저 재료 + 연관성 재료 토대로 레시피 검색 위해)
        
        for j, re in enumerate(recipes): # 레시피 인덱스, 레시피 반복
            if (add_ing[0] in re and add_ing[1] in re and add_ing[2]): # 유저 재료, 연관성 재료 모두 레시피에 있다면
                add_recipe.append(j) # 유저 재료, 연관성 재료를 모두 포함한 레시피 인덱스를 리스트에 추가

    add_recipe = set(add_recipe) # 레시피를 집합으로 변환해 중복 제거

    recipe_name = [] # 레시피 이름을 담을 리스트
    
    for i in add_recipe: # 레시피 인덱스 반복
        recipe_name.append(lbls[i]) # 라벨(레시피 타이틀) 인덱싱해서 타이틀을 저장
    
    return recipe_name

In [114]:
# def find_recipe(ingredients, rules, lbls):
#     rules_dic = {}
#     for i, item in enumerate(np.array(rules)[:, :-1]):
#         item = [list(i) for i in item]
#         rules_dic[str(item)] = lbls[i]
    
#     result = None
    
#     try:
#         result = rules_dic[str(ingredients)]
#     except:
#         result = "Recipe not found"
        
#     return result

In [115]:
print('Try to find recipe using ingredients: {0} and {1}'.format(user[0], user[1]))
found_recipe = find_many_recipe(user, rules, recipes, lbls)
print('Found recipe: ', found_recipe)

Try to find recipe using ingredients: 김치 and 대파
Found recipe:  ['불고기 토핑밥', '제육양념밥', '불고기 김치콩나물밥', '얼큰 김치우동', '참치김치찌개', '만두전골', '새우젓비지찌개', '차돌박이청국장', '두부김치찌개', '쌈장목살찌개', '쌈장김치찌개', '순두부김치찌개', '비지찌개', '부대라볶이', '닭갈비', '콩불', '삼겹살묵은지찜', '얼큰만제비', '김치스팸라면', '김치칼국수', '메밀전병']


In [64]:
print('Try to find recipe using ingredients: {0} and {1}'.format("간장", "소금"))
found_recipe = find_recipe([["다진마늘"], ["소금"]], rules, lbls)
print('Found recipe: ', found_recipe)

Try to find recipe using ingredients: 간장 and 소금
Found recipe:  Recipe not found


In [61]:
print('Try to find recipe using ingredients: {0} and {1}'.format("올리브유", "소금"))
found_recipe = find_recipe([["올리브유"], ["소금"]], rules, lbls)
print('Found recipe: ', found_recipe)

Try to find recipe using ingredients: 올리브유 and 소금
Found recipe:  Recipe not found


In [62]:
ingredients_to_try = [['소금'], ['후춧가루']]
print('Testing not finding ingredients')
print('Using the following ingredients: {0} and {1}'.format(ingredients_to_try[0][0], ingredients_to_try[1][0]))
print(find_recipe(ingredients_to_try, rules, lbls))

Testing not finding ingredients
Using the following ingredients: 소금 and 후춧가루
지단 김밥
